In [ ]:
!pip install Keras-Preprocessing
!pip install keras
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import numpy as np


import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
snowBallStemmer = SnowballStemmer("english")

import gensim
from gensim.models import Word2Vec


from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_colwidth', 1000)


import itertools
import string
import re
import time
import logging
import pickle

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
all_df=pd.read_csv('message.csv')
all_df.head()

In [ ]:
all_df=all_df.drop(['Compound','Negative','Positive'],axis=1)
all_df.head()

In [ ]:
all_df.info()

In [ ]:
all_df['message']=all_df['Message'].astype(str)

In [ ]:
#preprocessing the data
#Remove punctuations
#Tokenization - Converting a sentence into list of words
#Remove stopwords
#Lammetization/stemming 

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    stopword = stopwords.words('english')
    message_stopword=['today','day','still','want','going','got','now','quot','know','think','amp','really','twitter','need','see','im','hi','hey','am','rn','rt','tomorrow','day','night','pm']
    stopword+=message_stopword
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopword]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


all_df['message']=all_df['Message'].map(lambda s:preprocess(s))
all_df.head()

In [ ]:
all_df=all_df.drop(all_df[all_df.message.str.len()<1].index)
#dropping rows with empty string

In [ ]:
all_df.head()

In [ ]:
all_df.info()

In [ ]:
%timeit -n 100 all_df.message.str.len().max()

In [ ]:
all_df.head(100)

In [ ]:
decode_map = {0: "NEGATIVE",1: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]

In [ ]:
all_df['Sentiment'] = pd.to_numeric(all_df['Sentiment'], errors='coerce')

In [ ]:
all_df = all_df.dropna(subset=['Sentiment'])

In [ ]:
all_df['Sentiment'] = all_df['Sentiment'].astype(int)

In [ ]:
%%time
all_df.sentiment = all_df.Sentiment.apply(lambda x: decode_sentiment(x))

In [ ]:
from collections import Counter
sentiment_cnt = Counter(all_df.sentiment)

plt.figure(figsize=(16,8))
plt.bar(sentiment_cnt.keys(), sentiment_cnt.values())
plt.title("Dataset labels distribuition")

In [ ]:
TRAIN_SIZE = 0.8

# # TEXT CLENAING
# TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
SENTIMENT_THRESHOLDS = (0.5)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [ ]:
df_train, df_test = train_test_split(all_df, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

In [ ]:
%%time
documents = [_text.split() for _text in df_train.message]

In [ ]:
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE,
                                            window=W2V_WINDOW,
                                            min_count=W2V_MIN_COUNT,
                                            workers=8)

In [ ]:
w2v_model.build_vocab(documents)

In [ ]:
words = w2v_model.wv.key_to_index
vocab_size = len(words)
print("Vocab size", vocab_size)

In [ ]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

In [ ]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.message)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

In [ ]:
%%time
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.message), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.message), maxlen=SEQUENCE_LENGTH)

In [ ]:
#Label Encoder
labels = df_train.Sentiment.unique().tolist()
labels

In [ ]:
encoder = LabelEncoder()
encoder.fit(df_train.Sentiment.tolist())

y_train = encoder.transform(df_train.Sentiment.tolist())
y_test = encoder.transform(df_test.Sentiment.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

In [ ]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

In [ ]:
y_train[:10]

In [ ]:
#Embedding Layer
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

In [ ]:
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], input_length=SEQUENCE_LENGTH, trainable=False)

In [ ]:
#Build Model
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
#compile model

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [ ]:
%%time
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

In [ ]:
##Evaluate
acclstm = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print()
print("ACCURACY:",acclstm[1])

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()

plt.show()

plt.savefig('training_val_acc.png')

In [ ]:
def plot_confusion_matrix(cm, classes,title='Confusion matrix',
                             cmap=plt.cm.Blues):

   # This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.


    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
X = all_df.message # get all the text in x variable
y = all_df.sentiment # get all the sentiments into y variable
print(X.shape) #print shape of x
print(y.shape) # print shape of y

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=30) # split the data into traing and testing set where ratio is 80:20


In [ ]:
vect = TfidfVectorizer(analyzer = "word", ngram_range=(1,3)) # Get Tf-idf object and save it as vect. We can select features from here we just have simply change
                                                                                     #the ngram range to change the features also we can remove stop words over here with the help of stop parameter

In [ ]:
vect.fit(X_train) # fit or traing data to vect
X_train_dtm = vect.transform(X_train) # transform our training data 

In [ ]:
X_test_dtm = vect.transform(X_test)# transform our testing data 

In [ ]:
from sklearn.svm import LinearSVC # import SVC model from sklearn.svm
svm_clf = LinearSVC(random_state=0) # get object of SVC model with random_state parameter = 0

In [ ]:
svm_clf.fit(X_train_dtm, y_train)# fit our both traing data as well as its sentiments to the SVC model

In [ ]:
from sklearn.model_selection import cross_val_score  # import cross_val_score from sklear.model_selection
accuracies = cross_val_score(estimator = svm_clf, X = X_train_dtm, y = y_train, cv = 10)# do K- fold cross validation on our traing data and its sentimenst with 10 fold cross validation
accuracies.mean() # measure the mean accuray of 10 fold cross validation

In [ ]:
y_pred_svm = svm_clf.predict(X_test_dtm)  # predict the sentiments of testing data 

In [ ]:
from sklearn import metrics  # import metrics from sklearn
metrics.accuracy_score(y_test, y_pred_svm)  # measure the accuracy of our model on the testing data

In [ ]:
from sklearn.metrics import confusion_matrix # import confusion matrix from the sklearn.metrics
conf_matrix=confusion_matrix(y_test, y_pred_svm)# plot the confusion matrix between our predicted sentiments and the original testing data sentiments

In [ ]:
%%time
plt.figure(figsize=(6,6))
plot_confusion_matrix(conf_matrix, classes=df_train.Sentiment.unique(), title="Conf_matrix_svm")
plt.show()
plt.savefig('confusion_matrix.png')

In [ ]:
print(classification_report(y_test, y_pred_svm))